this notebook is intended for detecting and correcting fixed horizontal stripes that arise from buffer relay errors

# load packages

In [ ]:
import cv2
import numpy as np
import os
import fnmatch
import shutil
from matplotlib import pyplot as plt
from scipy.signal import find_peaks, periodogram

In [ ]:
def detect_stripes(frame, thresh=5):
    y = frame.mean(axis=1)
    f, Pxx_spec = periodogram(y, 1)
    peaks = find_peaks(np.sqrt(Pxx_spec), height=thresh)[0]
    return any(f[peaks] > 0.03)

def pad_frame(frame, video_cropped):
    fullframe = np.zeros((608,608), dtype='uint8') #default Miniscope frame size
    fullframe[video_cropped['h'][0]:video_cropped['h'][1], video_cropped['w'][0]:video_cropped['w'][1]] = frame
    return fullframe

def correct_stripes(frame, video_cropped, bufsize, buffertofix, offset):

    #pad croppped frames to equal original frame size
    if video_cropped is not None:
        frame = pad_frame(frame, video_cropped)

    #correct buffer
    flatframe = frame.flatten()
    buffermask = (np.arange(frame.shape[0] * frame.shape[1]) // bufsize % 2)+1 # labels first and second buffer
    originalbuffer = flatframe[buffermask==buffertofix] #extract buffer to fix
    fixedbuffer = np.roll(originalbuffer, offset)
    flatframe[buffermask==buffertofix] = fixedbuffer
    frame_fixed = flatframe.reshape((608,608))

    #remove padding
    if video_cropped is not None:
        frame_fixed = frame_fixed[video_cropped['h'][0]:video_cropped['h'][1], video_cropped['w'][0]:video_cropped['w'][1]]
    return frame_fixed

# set parameters

In [ ]:
stripe_thresh = 5

#if any cropping was applied to Miniscope, enter here. alternatively, set to None
#video_croped = None
video_cropped = {'h' : (0,600), 'w' : (0,600)} 

#below parameters should remain the same, but could potentially test different offsets
bufsize = 8184
buffertofix = 1 #1 or 2
offset = -8184 

# test correction on individual frames from single file

In [ ]:
f = '/Users/zachpennington/Desktop/TestVid/Bumjin/7.avi'
framenumber = 0

In [ ]:
cap = cv2.VideoCapture(f)
cap.set(cv2.CAP_PROP_POS_FRAMES, framenumber) 
ret, frame = cap.read() 
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

if detect_stripes(frame, thresh=stripe_thresh):
    frame_fixed = correct_stripes(frame, video_cropped, bufsize, buffertofix, offset)
    plt.figure()
    _, axarr = plt.subplots(1,2) 
    axarr[0].imshow(frame)
    axarr[1].imshow(frame_fixed)
else:
    print('no stripes detected')
    plt.imshow(frame)

_=cv2.waitKey(1)
cap.release()

# stripe correct entire video

below, all of the avi files for a recording session are rewritten in a new folder.  
nothing will be over-written

In [ ]:
directory = '/Users/zachpennington/Desktop/TestVid/Bumjin/' #make sure no backslash at very end

new_directory = os.path.join(os.path.split(directory)[0], '{}_destriped'.format(os.path.split(directory)[1]))
if not os.path.exists(new_directory):
    os.mkdir(new_directory)

files = fnmatch.filter(os.listdir(directory), '*avi')

for file in files:

    cap = cv2.VideoCapture(os.path.join(directory, file))  
    
    print('checking {} for striped frames'.format(file))
    cap_max = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    bad_frames = False    
    for f in range(cap_max):
        ret, frame = cap.read() 
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            stripes = detect_stripes(frame, thresh=stripe_thresh)
            if stripes:
                bad_frames = True
                break

    if bad_frames:
        print ('bad frames detected. beginning video re-write...')
        writer = cv2.VideoWriter(
            os.path.join(new_directory, file), 
            cv2.VideoWriter_fourcc(*'FFV1'), 30.0, (frame.shape[1], frame.shape[0]), isColor=False)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
        for f in range(cap_max):
            ret, frame = cap.read() 
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                if detect_stripes(frame, thresh=stripe_thresh):
                    frame = correct_stripes(frame, video_cropped, bufsize, buffertofix, offset)
                writer.write(frame)
        writer.release()
        
    else:
        print ('no bad frames dectected. copying original video to new location..')
        shutil.copy2(
            os.path.join(directory, file),
            os.path.join(new_directory, file)
        )
    
    cap.release()
    print('complete\n')